In [22]:
import os
import psycopg2

from dotenv import load_dotenv
# Le .env est aussi à la racine
load_dotenv(".env")

USER_PSQL = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD_PSQL = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

conn = psycopg2.connect(user=USER_PSQL, password=PASSWORD_PSQL,dbname="app_auth", host="localhost", port="5432")
conn.autocommit=True

In [17]:
from psycopg2 import sql
import timeit
import random

try:

    cur = conn.cursor()
    cur.execute("""
        SELECT user_id 
        FROM user_table
        LIMIT 100;
    """)
    users100 = cur.fetchall()

    def trigger_auto(users, counter):
        for user in users:
            for _ in range(counter):
                cur.execute(f"""
                    UPDATE user_table 
                    SET username = username || '{random.randint(1, 9)}' 
                    WHERE user_id = '{user[0]}';
                """)

    
    print("Question 4.3 avec trigger :")
    print("Fonction pour update 100 users :")   
    temp_user_100 =  timeit.timeit(lambda: trigger_auto(users100, 1),number=1)
    print(f"Temps d'éxecution  : {temp_user_100} \n")

    print("Fonction pour update 1000 users :")
    temp_user_1000 = timeit.timeit(lambda: trigger_auto(users100, 10),number=1)
    print(f"Temps d'éxecution  : {temp_user_1000} \n")

    print("Fonction pour update 10000 users :")
    temp_user_10000 =timeit.timeit(lambda: trigger_auto(users100, 100),number=1)
    print(f"Temps d'éxecution  : {temp_user_10000} \n")

    # Fermeture de la connexion
    cur.close()
except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL database: {e}")

Question 4.3 avec trigger :
Fonction pour update 100 users :
Temps d'éxecution  : 0.01941570000781212 

Fonction pour update 1000 users :
Temps d'éxecution  : 0.1744456999877002 

Fonction pour update 10000 users :
Temps d'éxecution  : 1.8638341000041692 



In [23]:
from psycopg2 import sql
import timeit
import random

try:

    cur = conn.cursor()
    cur.execute("""
        SELECT user_id
        FROM user_table
        LIMIT 100;
    """)
    users100 = cur.fetchall()

    print("Desactivation du trigger")
    cur.execute("""
        ALTER TABLE user_table DISABLE TRIGGER trigger_update_username ;
    """)

    def trigger_auto(users, counter):
        for user in users:
            for _ in range(counter):
                # On récupère l'username actuel, on ne l'a pas mis dans la requête au dessus car on fait X update sur l'user
                cur.execute(f"""
                    SELECT username 
                    FROM user_table
                    WHERE user_id = '{user[0]}';
                """)

                username = str(cur.fetchone())[2:-3]
                username = f"{username}{str(random.randint(1, 9))}"
                cur.execute(f"""
                    UPDATE user_table 
                    SET username = '{username}'
                    WHERE user_id = '{user[0]}';
                """)
                cur.execute(f"""
                    INSERT INTO username_history (user_id, username_new) 
                    VALUES({user[0]}, '{username}');
                """)

    
    print("Question 4.3 sans trigger :")
    print("Fonction pour update 100 users :")   
    temp_user_100 =  timeit.timeit(lambda: trigger_auto(users100, 1),number=1)
    print(f"Temps d'éxecution  : {temp_user_100} \n")

    print("Fonction pour update 1000 users :")
    temp_user_1000 = timeit.timeit(lambda: trigger_auto(users100, 10),number=1)
    print(f"Temps d'éxecution  : {temp_user_1000} \n")

    print("Fonction pour update 10000 users :")
    temp_user_10000 =timeit.timeit(lambda: trigger_auto(users100, 100),number=1)
    print(f"Temps d'éxecution  : {temp_user_10000} \n")

    # Fermeture de la connexion
    cur.close()
    conn.close()
except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL database: {e}")

Desactivation du trigger
Question 4.3 sans trigger :
Fonction pour update 100 users :
Temps d'éxecution  : 0.03525289999379311 

Fonction pour update 1000 users :
Temps d'éxecution  : 0.39085029999841936 

Fonction pour update 10000 users :
Temps d'éxecution  : 3.3489303999958793 

